In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import folium
from haversine import haversine, Unit
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 22.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [20, 6]

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('data/ID있는버전.xlsx')
df.head()

,ID,시군구,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,거래유형
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,201801,3,130000,4,1987,언주로 3,-
1,서울특별시 동작구 신대방동 롯데관악타워 150,서울특별시 동작구 신대방동,롯데관악타워,150,201801,4,74000,27,1996,보라매로5길 51,-
2,서울특별시 동작구 신대방동 경남교수 80,서울특별시 동작구 신대방동,경남교수,80,201801,25,65000,17,2001,여의대방로10길 14,-
3,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 80,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,80,201801,27,85800,15,2013,상도로 346-2,-
4,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 120,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,120,201801,24,104500,12,2013,상도로 346-2,-


In [3]:
df['계약년월'] = pd.to_datetime(df['계약년월'], format='%Y%m')
df['분기'], df['년도'] = (df['계약년월'].dt.quarter, df['계약년월'].dt.year)
df['분기'] = df[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
del df['년도']

In [4]:
df.head()

,ID,시군구,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,거래유형,분기
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,2018-01-01,3,130000,4,1987,언주로 3,-,2018/1
1,서울특별시 동작구 신대방동 롯데관악타워 150,서울특별시 동작구 신대방동,롯데관악타워,150,2018-01-01,4,74000,27,1996,보라매로5길 51,-,2018/1
2,서울특별시 동작구 신대방동 경남교수 80,서울특별시 동작구 신대방동,경남교수,80,2018-01-01,25,65000,17,2001,여의대방로10길 14,-,2018/1
3,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 80,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,80,2018-01-01,27,85800,15,2013,상도로 346-2,-,2018/1
4,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 120,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,120,2018-01-01,24,104500,12,2013,상도로 346-2,-,2018/1


In [8]:
cnt = df.groupby(['분기','ID']).count()
cnt = cnt.iloc[:,1:2]
cnt

단지명
분기     ID                                  
2018/1 서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80    4
       서울특별시 강남구 개포동 개포우성3차 160           2
       서울특별시 강남구 개포동 개포자이 130             2
       서울특별시 강남구 개포동 개포자이 170             2
       서울특별시 강남구 개포동 개포주공1단지 40          19
...                                     ...
2022/2 서울특별시 중랑구 신내동 화성.두산아파트 70          1
       서울특별시 중랑구 중화동 범양프레체 20             3
       서울특별시 중랑구 중화동 청원다미소 80             3
       서울특별시 중랑구 중화동 한영 80                1
       서울특별시 중랑구 중화동 현대휴앤미 80             1

[98286 rows x 1 columns]

In [12]:
cnt1 = cnt.loc['2018/1']
cnt2 = cnt.loc['2018/2']
cnt3 = cnt.loc['2018/3']
cnt4 = cnt.loc['2018/4']
cnt5 = cnt.loc['2019/1']
cnt6 = cnt.loc['2019/2']
cnt7 = cnt.loc['2019/3']
cnt8 = cnt.loc['2019/4']
cnt9 = cnt.loc['2020/1']
cnt10 = cnt.loc['2020/2']
cnt11 = cnt.loc['2020/3']
cnt12 = cnt.loc['2020/4']
cnt13 = cnt.loc['2021/1']
cnt14 = cnt.loc['2021/2']
cnt15 = cnt.loc['2021/3']
cnt16 = cnt.loc['2021/4']
cnt17 = cnt.loc['2022/1']
cnt18 = cnt.loc['2022/2']

In [14]:
co = pd.concat([cnt1,cnt2,cnt3,cnt4,cnt5,cnt6,cnt7,cnt8,cnt9,cnt10,cnt11,cnt12,cnt13,cnt14,cnt15,cnt16,cnt17,cnt18], axis=1)
co.to_excel('data/거래량.xlsx')